In [1]:
with open('23a.txt') as f:
    lines = f.readlines()

In [2]:
from typing import List, Tuple

Alright so I think my current gameplan is to take the board state, and for each pod, take every position in the board.
Then filter: if the end position is occupied, in front of doorway, is in wrong home, or there is a foreigner in the home that it is trying to go into, remove that position from the list.
if there is another pod in the way, remove it from the list.
if the pod is in the hallway, and the end position is also in the hallway, remove it from the list.
also probably remove it from the list if the end position is the upper home pos and the bottom home pos is free.

after you remove all invalid moves, make a new board state and increment its energy accordingly. Recursively check I guess? if no moves are possible or the end goal is achieved, stop. gotta keep track of the energies. after all possibilities have been checked, find the one with lowest energy.


In [3]:
points_dict = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

In [4]:
A_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,3),(3,3)]
B_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,5),(3,5)]
C_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,7),(3,7)]
D_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,9),(3,9)]

In [5]:
def base_grid():
    grid = [['.']*13 for i in range(5)]
    grid[0] = ['#'] * 13
    grid[1][0] = '#'
    grid[1][12] = '#'
    grid[2][0:3] = '#' * 3
    grid[2][10:13] = '#' * 3
    grid[2][4] = '#'
    grid[2][6] = '#'
    grid[2][8] = '#'
    grid[3][2] = '#'
    grid[3][4] = '#'
    grid[3][6] = '#'
    grid[3][8] = '#'
    grid[3][10] = '#'
    grid[4][2:11] = '#' * 9
    return grid

In [11]:
class Game_State:
    
    def __init__(self, pods: List[Tuple[int,int,chr]], energy: int):
        self.pods = pods
        self.base = base_grid()
        for coords in self.pods:
            row,col,ch = coords
            self.base[row][col] = ch
        self.energy = energy

    def check_point(self, row: int, col: int) -> bool:
        if self.base[row][col] != '.':
            return False
        return True

    def check_home(self, flavour: chr) -> bool:
        if flavour == 'A':
            if (not (self.base[2][3] == '.' or self.base[2][3] == flavour)) or (not (self.base[3][3] == '.' or self.base[3][3] == flavour)):
                return False
        if flavour == 'B':
            if (not (self.base[2][5] == '.' or self.base[2][5] == flavour)) or (not (self.base[3][5] == '.' or self.base[3][5] == flavour)):
                return False
        if flavour == 'C':
            if (not (self.base[2][7] == '.' or self.base[2][7] == flavour)) or (not (self.base[3][3] == '.' or self.base[3][3] == flavour)):
                return False
        if flavour == 'D':
            if (not (self.base[2][9] == '.' or self.base[2][9] == flavour)) or (not (self.base[3][9] == '.' or self.base[3][9] == flavour)):
                return False
        return True
         
    def __str__(self):
        string = ''
        for i in range(len(self.base)):
            string += str(self.base[i])
            if i != len(self.base) - 1: string += '\n'
        return string
    


In [7]:
init_pods = []
for row in range(len(lines)):
    for col in range(len(lines[row])):
        if lines[row][col] == 'A':
            init_pods.append((row,col,'A'))
        elif lines[row][col] == 'B':
            init_pods.append((row,col,'B'))
        elif lines[row][col] == 'C':
            init_pods.append((row,col,'C'))
        elif lines[row][col] == 'D':
            init_pods.append((row,col,'D'))
init_state = Game_State(init_pods)

In [9]:
def valid_end(goal: Tuple[int, int], flavour: chr, hallway: bool, grid: Game_State) -> bool:
        # check if not stopping on foreign home or in front of doorway.
        if flavour == 'A' and (row, col) not in A_spots:
            return False
        elif flavour == 'B' and (row, col) not in B_spots:
            return False
        elif flavour == 'C' and (row, col) not in C_spots:
            return False
        elif flavour == 'D' and (row, col) not in D_spots:
            return False
        
        #check if room doesnt contain any foreigners
        if row > 1 and not grid.check_home(flavour):
            return False

        #check that if the bottom home slot if open, the upper one is invalid. right?
        if row == 2 and grid.base[3][col] == '.':
            return False
        
        return True

In [12]:
def valid_path(start: Tuple[int, int], goal: Tuple[int, int], grid: Game_State) -> bool:
    if start[0] == 1 and goal[0] == 1: return False
    curr_row = start[0]
    curr_col = start[1]
    # check if have to move up first
    if goal[0] < start[0]:
        for i in range(start[0] - goal[0] + 1):
            if not grid.check_point(start[0] - i, start[1]):
                return False
        curr_row = goal[0]

    # check moving right
    if goal[1] > start[1]:
        for i in range(goal[1] - start[1] + 1):
            if not grid.check_point(curr_row, start[1] + i):
                return False
        curr_col = goal[1]
    #check moving left
    if goal[1] < start[1]:
        for i in range(start[1] - goal[1] + 1):
            if not grid.check_point(curr_row, start[1] - i):
                return False
        curr_col = goal[1]
    
    #check moving down after
    if goal[0] > start[0]:
        for i in range(goal[0] - start[0] + 1):
            if not grid.check_point(start[0] + i, curr_col):
                return False
        curr_row = goal[0]
    
    return True

        

In [ ]:
init_state